In [1]:
import pandas

results = pandas.read_csv('tmp/survey_results_public.csv', index_col=0)

In [2]:
cz_sk_selector = results['Country'].isin( ['Czech Republic', 'Slovak Republic'])

# preselect salaries
salaries = results['Salary']
salaries = salaries.dropna()

have_worked_language = results['HaveWorkedLanguage'].dropna()
have_worked_python =  have_worked_language.str.contains('Python')

want_work_language = results['WantWorkLanguage'].dropna()
want_work_python =  want_work_language.str.contains('Python')


In [3]:
print('Kolik lidí celkem vyplnilo dotazník?')
print(len(results))

Kolik lidí celkem vyplnilo dotazník?
51392


In [4]:
print('Kolik z nich bylo z ČR/SR?')
print(len(results[ cz_sk_selector ] ))

Kolik z nich bylo z ČR/SR?
581


In [5]:
print('Jaká je, podle dotazníku, průměrná mzda programátorů v ČR/SR? Ve světě?')
print('V ČR/SR', salaries[ cz_sk_selector ].mean())
print('Ve světe', salaries.mean())
print('Ve světe', salaries[ ~cz_sk_selector ].mean(), '(mimo ČR/SR)')

Jaká je, podle dotazníku, průměrná mzda programátorů v ČR/SR? Ve světě?
V ČR/SR 21211.71211
Ve světe 56298.4806406
Ve světe 56578.3188133 (mimo ČR/SR)


In [6]:
print('Kolik lidí z ČR/SR uvedlo, že používá* Python?')
print(len(results[cz_sk_selector & have_worked_python]))

Kolik lidí z ČR/SR uvedlo, že používá* Python?
129


In [7]:
print('Kolik z nich chce v používání* Pythonu pokračovat?')
print(len(results[cz_sk_selector & have_worked_python & want_work_python]))

Kolik z nich chce v používání* Pythonu pokračovat?
86
